In [1]:
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.ByteString                  as B
import qualified Data.ByteString.Char8            as B (lines)
import qualified Data.Attoparsec.ByteString       as A
import qualified Data.Attoparsec.ByteString.Char8 as AC

input <- B.lines <$> B.readFile "../input/day23.txt"

In [2]:
import Data.List

data Register = A | B | C | D deriving (Eq, Show, Ord)

data Instruction
    = Cpy (Either Int Register) Register
    | Inc Register
    | Dec Register
    | Jnz (Either Int Register) (Either Int Register)
    | Tgl Register
    | Invalid Instruction
    deriving (Eq, Show)

parseRegister = A.choice
    [ A.string "a" *> (return A)
    , A.string "b" *> (return B)
    , A.string "c" *> (return C)
    , A.string "d" *> (return D)
    ]

parseCpy = do
    A.string "cpy"
    AC.space
    value <- A.choice [Left <$> AC.signed AC.decimal, Right <$> parseRegister]
    AC.space
    reg <- parseRegister
    return $ Cpy value reg

parseInc = do
    A.string "inc"
    AC.space
    reg <- parseRegister
    return $ Inc reg

parseDec = do
    A.string "dec"
    AC.space
    reg <- parseRegister
    return $ Dec reg

parseJnz = do
    A.string "jnz"
    AC.space
    reg <- A.choice [Left <$> AC.signed AC.decimal, Right <$> parseRegister]
    AC.space
    jmp <- A.choice [Left <$> AC.signed AC.decimal, Right <$> parseRegister]
    return $ Jnz reg jmp

parseTgl = do
    A.string "tgl"
    AC.space
    reg <- parseRegister
    return $ Tgl reg

parseInstruction = A.choice [parseCpy, parseInc, parseDec, parseJnz, parseTgl]

parsed = map (either error id . A.parseOnly parseInstruction) input
mapM_ print parsed

Cpy (Right A) B
Dec B
Cpy (Right A) D
Cpy (Left 0) A
Cpy (Right B) C
Inc A
Dec C
Jnz (Right C) (Left (-2))
Dec D
Jnz (Right D) (Left (-5))
Dec B
Cpy (Right B) C
Cpy (Right C) D
Dec D
Inc C
Jnz (Right D) (Left (-2))
Tgl C
Cpy (Left (-16)) C
Jnz (Left 1) (Right C)
Cpy (Left 77) C
Jnz (Left 87) (Right D)
Inc A
Inc D
Jnz (Right D) (Left (-2))
Inc C
Jnz (Right C) (Left (-5))

In [3]:
-- import Control.Monad.Trans.State
import qualified Data.Map.Strict as M
import qualified Data.Vector     as V

type Registers = M.Map Register Int
data Computer = Computer
    { pointer      :: Int
    , registers    :: Registers
    } deriving (Eq, Show)

-- type Instructions = M.Map Int Instruction

runInstr (Computer p rs) (Cpy (Left value) reg) = let
    rs' = M.insert reg value rs
    in Computer (p+1) rs'
runInstr (Computer p rs) (Cpy (Right reg1) reg2) = let
    value = rs M.! reg1
    rs'   = M.insert reg2 value rs
    in Computer (p+1) rs'
runInstr (Computer p rs) (Inc reg) = let
    value = rs M.! reg
    rs'   = M.insert reg (value+1) rs
    in Computer (p+1) rs'
runInstr (Computer p rs) (Dec reg) = let
    value = rs M.! reg
    rs'   = M.insert reg (value-1) rs
    in Computer (p+1) rs'
runInstr (Computer p rs) (Jnz (Left value) jmp)
    | value == 0       = Computer (p+1) rs
    | (Left v)  <- jmp = Computer (p+v) rs
    | (Right r) <- jmp = let
        v = rs M.! r
        in               Computer (p+v) rs
runInstr (Computer p rs) (Jnz (Right reg) jmp)
    | (rs M.! reg) == 0 = Computer (p+1) rs
    | (Left v)  <- jmp = Computer (p+v) rs
    | (Right r) <- jmp = let
        v = rs M.! r
        in               Computer (p+v) rs

runTgl c@(Computer p rs) (Tgl reg) is = let
    value = rs M.! reg
    index = p + value
    len   = V.length is
    c'    = Computer (p+1) rs
    in if (index < 0 || index >= len)
        then (c', is)
        else (c', is V.// [(index, toggle (is V.! index))])


toggle instr = case instr of
    Inc x -> Dec x
    Dec x -> Inc x
    Tgl x -> Inc x
    Jnz a (Left b)  -> Invalid instr
    Jnz a (Right b) -> Cpy a b
    Cpy a b         -> Jnz a (Right b)
    Invalid i       -> toggle i

run c@(Computer p rs) instructions 
    | p >= V.length instructions = c
    | otherwise = let
        i  = instructions V.! p
        in case i of
            Tgl r -> let
                (c', instructions') = runTgl c i instructions
                in run c' instructions'
            _ -> let
                c' = runInstr c i
                in run c' instructions

start = Computer 0 $ M.fromList [(A, 7), (B, 0), (C, 0), (D, 0)]

a7 = registers (run start $ V.fromList parsed) M.! A
a7

11739

In [4]:
start' = Computer 0 $ M.fromList [(A, 9), (B, 0), (C, 0), (D, 0)]

makeStart n = Computer 0 $ M.fromList [(A, n), (B, 0), (C, 0), (D, 0)]

a n = registers (run (makeStart n) $ V.fromList parsed) M.! A

fact n = foldr (*) 1 [1..n]

7419 - fact 6

11739 - fact 7

47019 - fact 8

369579 - fact 9

3635499 - fact 10

39923499 - fact 11

6699

6699

6699

6699

6699

6699

In [5]:
fact 12 + 6699

479008299

In [6]:
-- Cpy (Left 77) C
-- Jnz (Left 87) (Right D)
77 * 87

6699